In [367]:
import pandas as pd
import numpy as np

def parse_parking_sheet(sheet_df):
    """
    Parse parking lot data into one row per date.
    Each block = 10 rows (or fewer if month ends early).
    Skips internal header rows inside the blocks.
    Ensures LOT 6 exists (set to -1 if missing).
    """
    rows = []
    n = len(sheet_df)
    i = 0
    first_month = None

    while i < n:
        block = sheet_df.iloc[i:i+10]
        if block.isna().all().all():
            i += 1
            continue

        # Row 0: Day and Date
        dummy_day = block.iloc[0, 0]
        date = block.iloc[0, 1]

        # Skip internal headers
        if isinstance(date, str) and "Number of Spaces" in date:
            i += 10
            continue

        # Parse date
        date = pd.to_datetime(date, errors='coerce')
        day = date.strftime("%A") if pd.notna(date) else None

        # Stop parsing if month changes
        if first_month is None and pd.notna(date):
            first_month = date.month
        elif pd.notna(date) and date.month != first_month:
            break

        entry = {"Day": day, "Date": date}

        # Rows 1-6 (or fewer) for LOTs
        for j in range(1, min(7, len(block))):
            lot_row = block.iloc[j]
            if pd.isna(lot_row[0]):
                continue
            lot_name = str(lot_row[0]).strip()

            spaces = pd.to_numeric(lot_row[2], errors="coerce") or 0
            vacancies = pd.to_numeric(lot_row[3], errors="coerce") or 0
            occupied = pd.to_numeric(lot_row[4], errors="coerce") or 0

            pct = lot_row[5]
            if isinstance(pct, str) and "%" in pct:
                pct = float(pct.replace("%", "").strip()) / 100
            pct = pd.to_numeric(pct, errors="coerce") or 0

            entry[f"{lot_name} Spaces"] = spaces
            entry[f"{lot_name} Vacancies"] = vacancies
            entry[f"{lot_name} Occupied"] = occupied
            entry[f"{lot_name} % Capacity"] = pct

        # Ensure LOT 6 exists
        if "LOT 6 Spaces" not in entry:
            entry["LOT 6 Spaces"] = -1
            entry["LOT 6 Vacancies"] = -1
            entry["LOT 6 Occupied"] = -1
            entry["LOT 6 % Capacity"] = -1

        # Totals (C/C and Cash) safely
        total_cc = 0
        total_cash = 0
        for j in range(1, min(7, len(block))):
            row = block.iloc[j]
            row_text = str(row[6]) if len(row) > 6 else ""
            value = pd.to_numeric(row[7], errors='coerce') if len(row) > 7 else 0
            value = value if not pd.isna(value) else 0

            if "C/C" in row_text:
                total_cc = value
            elif "Cash" in row_text:
                total_cash = value

        entry["Total CC Revenue"] = total_cc
        entry["Total Cash Revenue"] = total_cash
        entry["Total Revenue"] = total_cc + total_cash

        rows.append(entry)
        i += 10  # move to next block

    df = pd.DataFrame(rows)
    df.fillna(0, inplace=True)
    print("week added!" if rows else "⚠️ No rows parsed")
    return df


# ---------- READ AND COMBINE SECTION ----------
file_path = "CEOData/ONT Lot Counts 2020-2023/2020/PCI ONT Daily Lot Counts August 2020.xlsx"
#file_path = "CEOData/ONT Lot Counts 2020-2023/2020/PCI ONT Daily Lot Counts MAY 2020.xlsx"

try:
    all_sheets = pd.read_excel(file_path, sheet_name=None, header=None)
    print(f"✅ Loaded {len(all_sheets)} sheets.")

    # Only skip sheets literally named 'Summary' (case-insensitive)
    sheet_names = [name for name in all_sheets.keys() if str(name).lower() != "summary"]
    print("Sheets included:", sheet_names)

    b_df = pd.concat(all_sheets.values(), ignore_index=True)
    
    all_data = [parse_parking_sheet(all_sheets[name]) for name in sheet_names]
    combined_df = pd.concat(all_data, ignore_index=True)

    print("\n✅ Combined DataFrame created successfully!")
    print("Shape:", combined_df.shape)

except Exception as e:
    print("❌ Failed to read Excel file:")
    print(e)


✅ Loaded 6 sheets.
Sheets included: ['1-7', '8-14', '15-21', '22-28', '29 to end of the month']
week added!
week added!
week added!
week added!
week added!

✅ Combined DataFrame created successfully!
Shape: (31, 33)


In [402]:
import pandas as pd
import glob
import os

# Base folder where all yearly folders are
base_folder = "CEOData/ONT Lot Counts 2020-2023"

all_combined_data = []

for year in range(2020, 2024):  # 2020, 2021, 2022, 2023
    year_folder = os.path.join(base_folder, str(year))
    
    # Find all Excel files in that year's folder
    excel_files = glob.glob(os.path.join(year_folder, "*.xlsx"))
    
    print(f"✅ Found {len(excel_files)} files in {year_folder}")
    
    for file_path in excel_files:
        try:
            # Read all sheets without headers
            all_sheets = pd.read_excel(file_path, sheet_name=None, header=None)
            
            # Exclude last sheet if it's a summary
            sheet_names = list(all_sheets.keys())[:-1]
            
            # Parse each sheet
            for name in sheet_names:
                df = parse_parking_sheet(all_sheets[name])
                all_combined_data.append(df)
            
            print(f"Processed file: {file_path}")
            
        except Exception as e:
            print(f"❌ Failed to process {file_path}: {e}")

# Combine everything into one big DataFrame
combined_df = pd.concat(all_combined_data, ignore_index=True)
combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
combined_df = combined_df.sort_values('Date').reset_index(drop=True)

print("\n✅ All years combined successfully!")
print(combined_df.head())
print("Shape:", combined_df.shape)


✅ Found 8 files in CEOData/ONT Lot Counts 2020-2023\2020
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2020\PCI ONT Daily Lot Counts August 2020.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2020\PCI ONT Daily Lot Counts December 2020.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2020\PCI ONT Daily Lot Counts JULY 2020.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2020\PCI ONT Daily Lot Counts JUNE 2020.xlsx
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2020\PCI ONT Daily Lot Counts MAY 2020.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2020\PCI ONT Daily Lot Counts November 2020.xlsx


C:\Users\benja\AppData\Local\Temp\ipykernel_43392\3074141476.py:93: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.fillna(0, inplace=True)


week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2020\PCI ONT Daily Lot Counts October 2020.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2020\PCI ONT Daily Lot Counts September 2020.xlsx
✅ Found 12 files in CEOData/ONT Lot Counts 2020-2023\2021


C:\Users\benja\AppData\Local\Temp\ipykernel_43392\3074141476.py:93: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.fillna(0, inplace=True)


week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2021\PCI ONT Daily Lot Counts April 2021.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2021\PCI ONT Daily Lot Counts August 2021.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2021\PCI ONT Daily Lot Counts December 2021.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2021\PCI ONT Daily Lot Counts February 2021.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2021\PCI ONT Daily Lot Counts January 2021.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2021\PCI ONT Daily Lot Counts July 2021.xlsx
week added!
week added!
week added!
week added!
week added!
Pr

C:\Users\benja\AppData\Local\Temp\ipykernel_43392\3074141476.py:93: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.fillna(0, inplace=True)


week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2022\1) PCI ONT Daily Lot Counts January 2022.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2022\10) PCI ONT Daily Lot Counts October 2022.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2022\11) PCI ONT Daily Lot Counts November 2022.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2022\12) PCI ONT Daily Lot Counts December 2022.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2022\2) PCI ONT Daily Lot Counts February 2022.xlsx
week added!
week added!
week added!
week added!
week added!
Processed file: CEOData/ONT Lot Counts 2020-2023\2022\3) PCI ONT Daily Lot Counts March 2022.xlsx
week added!
week added!
week added!


In [448]:
# Ensure 'Date' is datetime
combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')

# Filter rows in March
march_rows = combined_df[combined_df['Date'].dt.month == 3]

# Find all duplicate dates in March (keep all duplicates)
duplicate_dates = march_rows['Date'][march_rows['Date'].duplicated(keep=False)]

# Get full rows for those duplicates
march_duplicates_df = combined_df[combined_df['Date'].isin(duplicate_dates)]

# Keep original index as a column
march_duplicates_df = march_duplicates_df.reset_index().rename(columns={'index': 'OriginalRow'})

# Display
march_duplicates_df


,OriginalRow,Day,Date,LOT 2 General Spaces,LOT 2 General Vacancies,LOT 2 General Occupied,LOT 2 General % Capacity,LOT 2 Premium Spaces,LOT 2 Premium Vacancies,LOT 2 Premium Occupied,...,LOT 5 Vacancies,LOT 5 Occupied,LOT 5 % Capacity,LOT 6 Spaces,LOT 6 Vacancies,LOT 6 Occupied,LOT 6 % Capacity,Total CC Revenue,Total Cash Revenue,Total Revenue
0,307,Monday,2021-03-01,1219,839,380.0,0.311731,324,220,104.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,36373.0,3772.0,40145.0
1,308,Monday,2021-03-01,1219,1219,0.0,0.000000,324,324,0.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,0.0,0.0,0.0
2,309,Tuesday,2021-03-02,1219,1219,0.0,0.000000,324,324,0.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,0.0,0.0,0.0
3,310,Tuesday,2021-03-02,1219,979,240.0,0.196883,324,289,35.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,8753.0,1190.0,9943.0
4,311,Wednesday,2021-03-03,1219,1219,0.0,0.000000,324,324,0.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,0.0,0.0,0.0
5,312,Wednesday,2021-03-03,1219,1009,210.0,0.172272,324,288,36.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,19509.0,1561.0,21070.0
6,313,Thursday,2021-03-04,1219,1219,0.0,0.000000,324,324,0.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,0.0,0.0,0.0
7,314,Thursday,2021-03-04,1219,1219,0.0,0.000000,324,324,0.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,26948.0,1773.0,28721.0
8,315,Friday,2021-03-05,1219,1219,0.0,0.000000,324,324,0.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,0.0,0.0,0.0
9,316,Friday,2021-03-05,1219,1219,0.0,0.000000,324,324,0.0,...,2200.0,0.0,0.000000,-1,-1,-1,-1,35757.0,1615.0,37372.0


In [450]:
#found out there are for 2021-2023, the first week of march has duplicates, and also there are 6 rows of 1970, and missing march 31, 2023
#but that last part is ok, take care of first two parts
#hard to remove, removing manually

# Suppose these are the row indices you want to remove
rows_to_remove = [308, 309, 311, 313, 315, 317, 319, 680, 681, 683, 686, 687, 689, 691, 1052, 1054, 1055, 1057, 1059, 1061, 1064]  # replace with your indices

# Drop them from the original DataFrame
combined_df = combined_df.drop(index=rows_to_remove)

# Optional: reset index if you want a clean 0…n-1 index
combined_df = combined_df.reset_index(drop=True)

rows_to_remove = combined_df[combined_df['Date'] == '1970-01-01'].index
combined_df = combined_df.drop(index=rows_to_remove).reset_index(drop=True)

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Day                       1062 non-null   object        
 1   Date                      1062 non-null   datetime64[ns]
 2   LOT 2 General Spaces      1062 non-null   int64         
 3   LOT 2 General Vacancies   1062 non-null   int64         
 4   LOT 2 General Occupied    1062 non-null   float64       
 5   LOT 2 General % Capacity  1062 non-null   float64       
 6   LOT 2 Premium Spaces      1062 non-null   int64         
 7   LOT 2 Premium Vacancies   1062 non-null   int64         
 8   LOT 2 Premium Occupied    1062 non-null   float64       
 9   LOT 2 Premium % Capacity  1062 non-null   float64       
 10  LOT 3 Spaces              1062 non-null   int64         
 11  LOT 3 Vacancies           1062 non-null   int64         
 12  LOT 3 Occupied      

In [458]:
#missing may 29-31 2020, and march 31 2023
combined_df.to_csv("all_parking_data.csv", index=False)